In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import uuid
import time

options = Options()
options.add_argument("--headless")  # Без отображения окна браузера
driver = webdriver.Chrome(options=options)

# Телефон беттерінің сілтемелері
product_urls = {
    "Toner mist": "https://kaspi.kz/shop/p/skin1004-mist-hyalu-cica-cloudy-120-ml-104904631/?0&c=750000000&ref=shared_link",
    "Dr.Althea 345": "https://kaspi.kz/shop/p/dr-althea-krem-345-relief-dlja-litsa-50-ml-115616909/?0&c=750000000&ref=shared_link",
    "Lagom": "https://kaspi.kz/shop/p/lagom-krem-cellus-mild-moisture-dlja-litsa-80-ml-100676372/?0&c=750000000&ref=shared_link",
    "telescopic тушь": "https://kaspi.kz/shop/p/tush-dlja-resnits-loreal-paris-telescopic-explosion-dlja-ob-ema-udlinjajuschaja-chernyi-17400312/?c=750000000&ref=shared_link",
    "Бальзам": "https://kaspi.kz/shop/p/fito-kosmetik-bal-zam-vazelin-uvlazhnenie-i-zaschita-10-102666500/?c=750000000&sr=6&qid=d4e7e5efef9988a52ecc9c155959dd20&ref=shared_link",
    "relouis пудра ": "https://kaspi.kz/shop/p/pudra-relouis-hd-powder-fixing-transparent-rassypchataja-01-belyi-100968225/?c=750000000&sr=1&qid=d285461498d0d5703498499264187614&ref=shared_link"
}

all_reviews = []  # Барлық пікірлерді сақтау үшін бос тізім

# Әрбір өнімнің атауы мен сілтемесін алып, бетке өтіп, пікірлерді жинаймыз
for product_name, url in product_urls.items():
    driver.get(url)  # Өнімнің бетіне өту
    time.sleep(5)  # Беттің толық жүктелуін күту

    soup = BeautifulSoup(driver.page_source, "html.parser")  # HTML құрылымды талдау
    reviews = soup.find_all("div", class_="reviews__review")  # Барлық пікірлерді табу

    count = 0  # Пікір санағыш
    for review in reviews:
        if count >= 50:  # Егер 50 пікірден асса, циклді тоқтату
            break

        review_id = str(uuid.uuid4())  # Әр пікірге бірегей ID беру

        # Пікір күні
        date_tag = review.find("div", class_="reviews__date")
        date = date_tag.text.strip() if date_tag else None

        # Пайдаланушы аты
        user_tag = review.find("div", class_="reviews__author")
        user = user_tag.text.strip() if user_tag else "Аноним"

        # Пікір мәтіні
        comment_tag = review.find("div", class_="reviews__review-text")
        comment = comment_tag.get_text(strip=True).replace("Комментарий:", "") if comment_tag else None

        # Бағалау (рейтинг) – мысалы: _40 -> 4.0
        rating_tag = review.find("div", class_=lambda c: c and "rating" in c)
        rating = None
        if rating_tag:
            classes = rating_tag.get("class")
            for cls in classes:
                if cls.startswith("_"):  # Кластың атауы "_" символымен басталса
                    try:
                        rating = int(cls[1:]) / 10  # Мысалы: "_40" -> 4.0
                    except ValueError:
                        rating = None

        # Пікірді сөздік түрінде тізімге қосамыз
        all_reviews.append({
            "ID": review_id,
            "Атауы": product_name,
            "Күні": date,
            "Пайдаланушы": user,
            "Баға": rating,
            "Пікір": comment
        })

        count += 1  # Келесі пікірге өту үшін санағышты арттырамыз

driver.quit()  # Браузерді жабу

# Жиналған пікірлерді DataFrame-ке айналдырып, CSV файлына сақтаймыз
df = pd.DataFrame(all_reviews)
df.to_csv("Kaspi_pikirleri.csv", index=False, encoding='utf-8-sig')
print("✅ пікір сәтті сақталды!")  # Процестің сәтті аяқталғанын хабарлау


✅ пікір сәтті сақталды!


In [30]:
df = pd.read_csv("Kaspi_pikirleri.csv")
df

,ID,Атауы,Күні,Пайдаланушы,Баға,Пікір
0,30413d14-73ba-4504-95d2-956bf19e5e58,Toner mist,17.06.2024,Ирина,5.0,Впервые пробую этот мист. Мне понравился. Осве...
1,28260584-9a26-40f2-b098-fe1813936ffb,Toner mist,28.02.2023,Наргиза,4.0,"Достоинства:Удобно брызгать, расход небольшой...."
2,cd2bf942-9453-4c18-b9f8-711ba66d3a67,Toner mist,23.09.2024,Нұрайлым,5.0,"Құрбымнан көріп алғанмын, бетім құрғақ болғасы..."
3,252a932f-a707-4586-b300-772ddffefdcd,Toner mist,13.07.2024,Ақерке,5.0,"Очень классная, качество супер, советую взять...."
4,4b9dcb29-5743-4df2-a357-373c8129ae26,Toner mist,24.03.2024,Гульзада,3.0,"Мист шашатын жері істен шыққан, брак, сатушыға..."
5,f395d024-62c2-4f99-8bb7-29b3fd19796e,Toner mist,14.04.2025,Рита,5.0,"Немного шиплет от того, что сразу даёт эффекты..."
6,a323412b-45c3-4a5b-a271-af2991c6cdd5,Toner mist,04.02.2025,Коркем,5.0,Вся серия Skin 1004 супер! Каспи как всегда лу...
7,1f6dde21-9551-4270-892a-d7b9b61f79d0,Toner mist,17.01.2025,Сымбат,5.0,Товар оригинал! Спасибо большое магазину Korea...
8,9bb0238d-aff5-487c-8bfa-43109baefc54,Toner mist,04.12.2024,Женіс,5.0,Күшті.0 человек(а) посчитал(и) отзыв полезным
9,849b9f23-cfa6-4049-993a-531b608b4e3a,Dr.Althea 345,15.02.2024,Лилия,5.0,"Самый лучший, лёгкий, не вызывает аллергию, пр..."


In [31]:
import re
from langdetect import detect

# Функция пікірдің тілін тексеру және қазақ/орыс тілін анықтау
def is_valid_language(text):
    try:
        language = detect(text)
        return language in ['kk', 'ru']  # Тек қазақ және орыс тілдерін қалдырамыз
    except:
        return False

# Пікірді тазарту: артық сөздер мен символдарды алып тастау
def clean_text(text):
    if not text:
        return ""
    # Артық символдарды алып тастау
    text = re.sub(r'\s+', ' ', text)  # Көп бос орынды бір бос орынмен ауыстыру
    text = re.sub(r'[^\w\s,.-]', '', text)  # Тек әріптер мен сандар, үтір, нүкте, сызықшалар қалдыру
    return text.strip()

# Тазарту процесі
df['Пікір'] = df['Пікір'].apply(lambda x: clean_text(x) if is_valid_language(x) else "")


In [32]:
# Пікір ұзындығын есептеу
df['Пікір Ұзындығы'] = df['Пікір'].apply(lambda x: len(str(x)) if x else 0)

# Баға бағанын сандық форматқа түрлендіру
df['Баға'] = pd.to_numeric(df['Баға'], errors='coerce')  # Қате мәндерді NaN деп қалдырады


In [33]:
# Деректерді CSV файлында сақтау
df.to_csv("Kaspi_pikirleri_cleaned.csv", index=False, encoding='utf-8-sig')

In [34]:
df = pd.read_csv("Kaspi_pikirleri_cleaned.csv")
df

,ID,Атауы,Күні,Пайдаланушы,Баға,Пікір,Пікір Ұзындығы
0,30413d14-73ba-4504-95d2-956bf19e5e58,Toner mist,17.06.2024,Ирина,5.0,Впервые пробую этот мист. Мне понравился. Осве...,238
1,28260584-9a26-40f2-b098-fe1813936ffb,Toner mist,28.02.2023,Наргиза,4.0,"ДостоинстваУдобно брызгать, расход небольшой.Н...",132
2,cd2bf942-9453-4c18-b9f8-711ba66d3a67,Toner mist,23.09.2024,Нұрайлым,5.0,"Құрбымнан көріп алғанмын, бетім құрғақ болғасы...",194
3,252a932f-a707-4586-b300-772ddffefdcd,Toner mist,13.07.2024,Ақерке,5.0,"Очень классная, качество супер, советую взять....",81
4,4b9dcb29-5743-4df2-a357-373c8129ae26,Toner mist,24.03.2024,Гульзада,3.0,"Мист шашатын жері істен шыққан, брак, сатушыға...",225
5,f395d024-62c2-4f99-8bb7-29b3fd19796e,Toner mist,14.04.2025,Рита,5.0,"Немного шиплет от того, что сразу даёт эффекты...",97
6,a323412b-45c3-4a5b-a271-af2991c6cdd5,Toner mist,04.02.2025,Коркем,5.0,Вся серия Skin 1004 супер Каспи как всегда луч...,85
7,1f6dde21-9551-4270-892a-d7b9b61f79d0,Toner mist,17.01.2025,Сымбат,5.0,Товар оригинал Спасибо большое магазину Korean...,171
8,9bb0238d-aff5-487c-8bfa-43109baefc54,Toner mist,04.12.2024,Женіс,5.0,Күшті.0 человека посчитали отзыв полезным,41
9,849b9f23-cfa6-4049-993a-531b608b4e3a,Dr.Althea 345,15.02.2024,Лилия,5.0,"Самый лучший, лёгкий, не вызывает аллергию, пр...",232
